In [114]:
import os
from dotenv import load_dotenv
import requests
import pandas as pd
import time

load_dotenv();

In [115]:
# utility to return (artist_id , song/album id )
def extract_ids(rows,arr):
    artists,id = rows
    for artist in artists:
        arr.append((artist['id'],id))

In [116]:
ENDPOINT = "https://api.spotify.com/v1/me/tracks"
HEADERS={f"Authorization":f"Bearer  {os.getenv("ACCESS_TOKEN")}"}
ALBUMS_PATH = os.getenv("ALBUMS_CSV")
ARTISTS_PATH = os.getenv("ARTISTS_CSV")
SONGS_PATH = os.getenv("SONGS_CSV")
ARTISTS_ALBUMS_PATH = os.getenv("ARTIST_ALBUMS_CSV")
ARTISTS_SONGS_PATH = os.getenv("ARTISTS_SONGS_CSV")

collection , art_song_pairs , alb_art_pairs = [] , [] , []
original_songs = pd.read_csv(SONGS_PATH,index_col=0)
original_artists = pd.read_csv(ARTISTS_PATH,index_col=0)

In [117]:
# API request to get all songs in 'liked songs'. Be care to ensure limit + offset get every song in the playlist
# Collection is a list of songs with all info attached
# i.e doesn't stop short
for offset in range(0,6):
    params = {'limit' : 49, 'offset' : offset*49}
    response = requests.get(ENDPOINT,headers=HEADERS,params=params)
    print(response.status_code)
    for x in response.json()['items']:
        collection.append(x['track'])

200
200
200
200
200
200


In [118]:
## Get Albums database
dfs_albums = [x['album'] for x in collection]
albums = pd.DataFrame(df for df in dfs_albums)
albums[['artists','id']].apply(lambda x : extract_ids(x,alb_art_pairs),axis=1) # for albums-artists
albums['external_urls'] = albums['external_urls'].apply(lambda x : x['spotify'])
albums.drop(columns=['artists'],inplace=True,axis=1)
albums.to_csv(path_or_buf = ALBUMS_PATH)

In [119]:
## Gets artist - album database
art_albs = pd.DataFrame(alb_art_pairs,columns=["ArtistId","AlbumId"])
art_albs.to_csv(path_or_buf = ARTISTS_ALBUMS_PATH)

In [120]:
# Gets artists database + adds on country
# Note has to work slightly different as deals with a lists of artists for each song
artists =  []
dfs_artists = [x['artists'] for x in collection]
for x in dfs_artists:
    for artist in x:
        artists.append(pd.DataFrame(artist))
artists_expanded = pd.concat(artists)


In [121]:
added_artists = pd.concat([original_artists, artists_expanded]).drop_duplicates(keep=False)
added_artists.drop_duplicates(subset=['id'],inplace=True)
added_artists = added_artists[added_artists['country'].isna()]
added_artists

,external_urls,href,id,name,type,uri,country


In [122]:
artists_list = added_artists['name'].to_list()
artists_ids = added_artists['id'].to_list()
COUNTRY_ENDPOINT = "https://musicbrainz.org/ws/2/artist"
COUNTRY_HEADERS = {'User-Agent': "spotify_data_analysis/1.0.0 annabelflett@gmail.com"}

places = []
for name in artists_list:
    params = {'query' : name,"limit" : 1,'fmt':'json'}
    response = requests.get(COUNTRY_ENDPOINT,headers=COUNTRY_HEADERS,params=params)
    try: 
        country = response.json()['artists'][0]['area']['name']
        places.append(country)
    except KeyError:
        places.append(None)
    time.sleep(1)

added_artists['country'] = places
z = pd.concat([original_artists,added_artists],axis=0).drop_duplicates()
z.to_csv(ARTISTS_PATH)

In [123]:
# Gets songs database
songs = pd.DataFrame(df for df in collection)
songs['album_id']= songs['album'].apply(lambda x : x['id'])
songs[['artists','id']].apply(lambda x : extract_ids(x,art_song_pairs),axis=1)
songs['external_urls'] = songs['external_urls'].apply(lambda x : x['spotify'])
songs['isrc'] = songs['external_ids'].apply(lambda x : x['isrc'])
songs.drop(columns=['album','artists','external_ids'],inplace=True,axis=1)
songs.to_csv(path_or_buf = SONGS_PATH);

In [124]:
added_songs = pd.concat([songs['id'],original_songs['id']]).drop_duplicates(keep=False).to_list()
added_songs

[]

In [125]:
# Gets artist - song database
artists_songs = pd.DataFrame(art_song_pairs,columns=["ArtistId","SongId"])
artists_songs.to_csv(path_or_buf = ARTISTS_SONGS_PATH)

In [126]:
# Update song_stats
if added_songs != []:
    print("adding songs")
    STATS_ENDPOINT = "https://api.reccobeats.com/v1/audio-features"
    STATS_HEADERS = {'Accept': 'application/json'}
    original_song_stats = pd.read_csv(os.getenv("SONG_STATS_CSV"),index_col=0)
    ids_chunked = [added_songs[i:i+40] for i in range(0,len(added_songs)+1,40)]
    df = []

    for song in ids_chunked:
        params = {'ids' : song}
        response = requests.get(STATS_ENDPOINT,headers=STATS_HEADERS,params=params)
        print(response.json())
        for x in response.json()['content']:
            df.append(x)
        


    song_stats = pd.DataFrame(df)
    try :
        song_stats['spotify_id'] = song_stats['href'].apply(lambda x : x[31:])
    except KeyError:
        pass
    song_stats = pd.concat([song_stats,original_song_stats],axis=0)

    x = pd.merge(left=songs,right=song_stats,left_on='id',right_on='spotify_id',how='left')
    na_ids = (x[x['spotify_id'].isna()]['id_x']).rename('spotify_id')
    song_stats = pd.concat([song_stats,na_ids],axis=0)
    song_stats.to_csv(os.getenv("SONG_STATS_CSV"))